In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [4]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [13]:
pd.read_sql(
    """
    SELECT * FROM boston
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


CRIM - уровень преступности на душу населения
ZN - доля земель под жилую застройку зонирована под участки площадью более 25 000 кв. футов.
INDUS - доля акров неторгового бизнеса на город.
CHAS - 1, если граничит с рекой, 0 иначе
NOX - концентрация оксидов азота (частей на 10 миллионов)
RM - среднее количество комнат в квартире
AGE - доля жилых домов, построенных до 1940 г.
DIS - взвешенные расстояния до пяти центров занятости Бостона
RAD - индекс доступности к радиальным магистралям
TAX - полная ставка налога на имущество за 10 000 долларов США
PTRATIO - соотношение учеников и учителей по городам
B - 1000(Bk - 0.63)^2, где Bk доля чернокожих людей по городам
LSTAT - % более низкого статуса населения
MEDV - Средняя стоимость домов в 1000 долларов.

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [100]:
pd.read_sql(
    """
    SELECT "CRIM " AS FIELD_NAME, count(*)-Count(CRIM) as NULL_VAL FROM boston
    UNION ALL
    SELECT "ZN " AS FIELD_NAME, count(*)-Count(ZN) as NULL_VAL FROM boston
    UNION ALL
    SELECT "INDUS " AS FIELD_NAME, count(*)-Count(INDUS) as NULL_VAL FROM boston
    UNION ALL
    SELECT "CHAS " AS FIELD_NAME, count(*)-Count(CHAS) as NULL_VAL FROM boston
    UNION ALL
    SELECT "NOX " AS FIELD_NAME, count(*)-Count(NOX) as NULL_VAL FROM boston
    """,
    con,
)

,FIELD_NAME,NULL_VAL
0,CRIM,0
1,ZN,0
2,INDUS,0
3,CHAS,0
4,NOX,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [102]:
pd.read_sql(
    """
    SELECT "CRIM " AS FIELD_NAME, Count(Distinct CRIM) as NULL_VAL FROM boston
    UNION ALL
    SELECT "ZN " AS FIELD_NAME, Count(Distinct ZN) as NULL_VAL FROM boston
    UNION ALL
    SELECT "INDUS " AS FIELD_NAME, Count(Distinct INDUS) as NULL_VAL FROM boston
    UNION ALL
    SELECT "CHAS " AS FIELD_NAME, Count(Distinct CHAS) as NULL_VAL FROM boston
    UNION ALL
    SELECT "NOX " AS FIELD_NAME, Count(Distinct NOX) as NULL_VAL FROM boston
    """,
    con,
)

,FIELD_NAME,NULL_VAL
0,CRIM,504
1,ZN,26
2,INDUS,76
3,CHAS,2
4,NOX,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [101]:
pd.read_sql(
    """
    with rows as(
    Select CRIM, row_number() OVER (ORDER BY CRIM) as CRIM_ROWS, ZN, row_number() OVER (ORDER BY ZN) as ZN_ROWS, INDUS, row_number() OVER (ORDER BY INDUS) as INDUS_ROWS, CHAS, row_number() OVER (ORDER BY CHAS) as CHAS_ROWS
    , NOX, row_number() OVER (ORDER BY NOX) as NOX_ROWS FROM boston)

    SELECT FIELD_NAME FROM(
        SELECT FIELD_NAME, VAL, LAG(VAL) OVER (PARTITION BY FIELD_NAME) AS VAL_1 FROM(
                Select "CRIM " AS FIELD_NAME, AVG(CRIM) as VAL FROM (Select CRIM from rows
                Where CRIM_ROWS	= 253 or CRIM_ROWS	= 254)
                UNION ALL
                Select "CRIM " AS FIELD_NAME,min(CRIM) as VAL  from rows
                UNION ALL
                Select "ZN " AS FIELD_NAME, AVG(ZN) as VAL FROM (Select ZN from rows
                Where ZN_ROWS	= 253 or ZN_ROWS	= 254)
                UNION ALL
                Select "ZN " AS FIELD_NAME,min(ZN) AS VAL  from rows
                UNION ALL
                Select "INDUS " AS FIELD_NAME, AVG(INDUS) as VAL FROM (Select INDUS from rows
                Where INDUS_ROWS	= 253 or INDUS_ROWS	= 254)
                UNION ALL
                Select "INDUS " AS FIELD_NAME,min(INDUS) AS VAL  from rows
                UNION ALL
                Select "CHAS " AS FIELD_NAME, AVG(CHAS) as VAL FROM (Select CHAS from rows
                Where CHAS_ROWS	= 253 or CHAS_ROWS	= 254)
                UNION ALL
                Select "CHAS " AS FIELD_NAME,min(CHAS) AS VAL  from rows   
                UNION ALL
                Select "NOX " AS FIELD_NAME, AVG(NOX) as VAL FROM (Select NOX from rows
                Where NOX_ROWS	= 253 or NOX_ROWS	= 254)
                UNION ALL
                Select "NOX " AS FIELD_NAME,min(NOX) AS VAL  from rows
        )
    )
    WHERE VAL=VAL_1
    """,
    con,
)

,FIELD_NAME
0,CHAS
1,ZN


Исходя только из той информации, что минимальное значение и медиана в данных полях равны, можно сделать вывод, что как минимум 254 значения в этих полях одинаковые. В случае с полем CHAS - как минимум 254 дома/участка(?) не граничат с рекой. В случае с ZN, зная, что минимальное значение в нем 0, можем сделать вывод, что большая часть участков не имеет выделенных под жилую застройку зон.

4. Выведите разницу между средним количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [138]:
pd.read_sql(
    """
    with data_set as(
    SELECT "25" as Field_type, AVG(RM) as ROOM_COUNT, AVG(MEDV) as PRICE FROM(SELECT MEDV,RM From boston Order by MEDV limit 25)
    UNION ALL
    SELECT "50" as Field_type, AVG(RM) as ROOM_COUNT, AVG(MEDV) as PRICE FROM(SELECT MEDV,RM From boston Order by MEDV limit 50)
    UNION ALL
    SELECT "100" as Field_type, AVG(RM) as ROOM_COUNT, AVG(MEDV) as PRICE FROM(SELECT MEDV,RM From boston Order by MEDV limit 100)
    UNION ALL
    SELECT "200" as Field_type, AVG(RM) as ROOM_COUNT, AVG(MEDV) as PRICE FROM(SELECT MEDV,RM From boston Order by MEDV limit 200)
    UNION ALL
    SELECT "300" as Field_type, AVG(RM) as ROOM_COUNT, AVG(MEDV) as PRICE FROM(SELECT MEDV,RM From boston Order by MEDV limit 300)
    )
    
    Select Field_type, ROOM_COUNT, ROOM_COUNT_EXP, ROOM_COUNT_EXP-ROOM_COUNT as DIFF, PRICE  FROM(
    Select * from data_set
    JOIN
    (SELECT AVG(RM) as ROOM_COUNT_EXP FROM boston WHERE MEDV=(SELECT max(MEDV) from boston)))
    """,
    con,
)

,Field_type,ROOM_COUNT,ROOM_COUNT_EXP,DIFF,PRICE
0,25,5.747840,7.484,1.736160,7.848000
1,50,5.753240,7.484,1.730760,9.650000
2,100,5.887120,7.484,1.596880,11.817000
3,200,5.911705,7.484,1.572295,14.842500
4,300,5.972227,7.484,1.511773,16.927667


Стоимость самых дорогих домов 50.000$, среднее количество комнат в этих домах 7,5. Средняя стоимость 25 самых дешевых домов составляет 7.800$ (на 85% дешевле), при этом количество комнат меньше на 1,7 шт (меньше на 25%). Разница в стоимости 300 самых дешевых домов и 25 самых дешевых домов составляет 9.100$, при этом разница в количестве комнат меняется не значительно - всего на 0,22. Можем сделать вывод, что количество комнат на стоимость влияет, но не так существенно, как другие параметры.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [147]:
pd.read_sql(
    """
    SELECT LSTAT, MEDV, rank() OVER (ORDER BY LSTAT DESC) as RANK_LSTAT FROM boston 
    WHERE MEDV=(SELECT max(MEDV) from boston)
    """,
    con,
)

,LSTAT,MEDV,RANK_LSTAT
0,9.53,50.0,1
1,8.88,50.0,2
2,7.44,50.0,3
3,5.12,50.0,4
4,4.63,50.0,5
5,4.45,50.0,6
6,3.73,50.0,7
7,3.70,50.0,8
8,3.32,50.0,9
9,3.26,50.0,10


Вывод: дорогие дома располагаются в районах, в которых процент населения с более низким статусом не превышает 10%). Самый низкий процент населения с более низким статусом - 1,73%.

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [151]:
pd.read_sql(
    """
    SELECT CHAS, AVG(MEDV) as AVR_PRICE FROM boston
    Group by CHAS
    """,
    con,
)

,CHAS,AVR_PRICE
0,0.0,22.093843
1,1.0,28.440000


Дома, граничащие с рекой, почти на 30% дороже.

7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [160]:
pd.read_sql(
    """
    with rows as(
    SELECT "CRIM " as FIELD_NAME, CHAS, AVG(CRIM) as AVG_VAL FROM boston WHERE CHAS = 0
    UNION ALL
    SELECT "CRIM " as FIELD_NAME, CHAS, AVG(CRIM) as AVG_VAL FROM boston WHERE CHAS = 1
    UNION ALL
    SELECT "ZN " as FIELD_NAME, CHAS, AVG(ZN) as AVG_VAL FROM boston WHERE CHAS = 0
    UNION ALL
    SELECT "ZN " as FIELD_NAME, CHAS, AVG(ZN) as AVG_VAL FROM boston WHERE CHAS = 1
    UNION ALL
    SELECT "INDUS " as FIELD_NAME, CHAS, AVG(INDUS) as AVG_VAL FROM boston WHERE CHAS = 0
    UNION ALL
    SELECT "INDUS " as FIELD_NAME, CHAS, AVG(INDUS) as AVG_VAL FROM boston WHERE CHAS = 1
    UNION ALL
    SELECT "NOX " as FIELD_NAME, CHAS, AVG(NOX) as AVG_VAL FROM boston WHERE CHAS = 0
    UNION ALL
    SELECT "NOX " as FIELD_NAME, CHAS, AVG(NOX) as AVG_VAL FROM boston WHERE CHAS = 1
    )

    Select Field_Name from(
        Select Field_name, Chas, AVG_VAL, LAG(AVG_VAL) OVER (Partition by Field_name) as AVG_NO_RIVER from rows
        )
    Where AVG_VAL-AVG_NO_RIVER>0

    """,
    con,
)

,Field_name
0,INDUS
1,NOX


Среднее значение выше, когда местность граничит с рекой у двух показателей - INDUS и NOX. Доля акров неторгового бизнеса на город(INDUS) выше, если местность граничит с рекой, возможно это связано с особенностями использования земель в водоохранных зонах. Концентрация оксидов азота также выше (NOX) в местностях с рекой. Возможно повышение двух этих параметров связано с размещением в зонах у рек различных заводов. 

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [215]:
pd.read_sql(
    """

    WITH NOX_PER as (SELECT DISTINCT "10" as PERC_VAL, first_value(NOX) OVER (Order BY CASE WHEN p<=0.1 THEN p END DESC) x
    FROM(SELECT NOX, PERCENT_RANK() over(Order by NOX) p FROM boston)
    UNION ALL
    SELECT DISTINCT "20" as PERC_VAL, first_value(NOX) OVER (Order BY CASE WHEN p<=0.2 THEN p END DESC) x
    FROM(SELECT NOX, PERCENT_RANK() over(Order by NOX) p FROM boston)
    UNION ALL
    SELECT DISTINCT "30" as PERC_VAL, first_value(NOX) OVER (Order BY CASE WHEN p<=0.3 THEN p END DESC) x
    FROM(SELECT NOX, PERCENT_RANK() over(Order by NOX) p FROM boston)
    UNION ALL
    SELECT DISTINCT "40" as PERC_VAL, first_value(NOX) OVER (Order BY CASE WHEN p<=0.4 THEN p END DESC) x
    FROM(SELECT NOX, PERCENT_RANK() over(Order by NOX) p FROM boston)
    UNION ALL
    SELECT DISTINCT "50" as PERC_VAL, first_value(NOX) OVER (Order BY CASE WHEN p<=0.5 THEN p END DESC) x
    FROM(SELECT NOX, PERCENT_RANK() over(Order by NOX) p FROM boston)
    UNION ALL
    SELECT DISTINCT "60" as PERC_VAL, first_value(NOX) OVER (Order BY CASE WHEN p<=0.6 THEN p END DESC) x
    FROM(SELECT NOX, PERCENT_RANK() over(Order by NOX) p FROM boston)
    UNION ALL
    SELECT DISTINCT "70" as PERC_VAL, first_value(NOX) OVER (Order BY CASE WHEN p<=0.7 THEN p END DESC) x
    FROM(SELECT NOX, PERCENT_RANK() over(Order by NOX) p FROM boston)
    UNION ALL
    SELECT DISTINCT "80" as PERC_VAL, first_value(NOX) OVER (Order BY CASE WHEN p<=0.8 THEN p END DESC) x
    FROM(SELECT NOX, PERCENT_RANK() over(Order by NOX) p FROM boston)
    UNION ALL
    SELECT DISTINCT "90" as PERC_VAL, first_value(NOX) OVER (Order BY CASE WHEN p<=0.9 THEN p END DESC) x
    FROM(SELECT NOX, PERCENT_RANK() over(Order by NOX) p FROM boston)
    UNION ALL
    SELECT DISTINCT "100" as PERC_VAL, first_value(NOX) OVER (Order BY CASE WHEN p<=1 THEN p END DESC) x
    FROM(SELECT NOX, PERCENT_RANK() over(Order by NOX) p FROM boston))

    
    Select NOX_PER.PERC_VAL, x as NOX_PERC, x1 as INDUS_PERC from NOX_PER
    JOIN
    (SELECT DISTINCT "10" as PERC_VAL, first_value(INDUS) OVER (Order BY CASE WHEN p<=0.1 THEN p END DESC) x1
    FROM(SELECT Indus, PERCENT_RANK() over(Order by Indus) p FROM boston)
    UNION ALL
    SELECT DISTINCT "20" as PERC_VAL, first_value(INDUS) OVER (Order BY CASE WHEN p<=0.2 THEN p END DESC) x1
    FROM(SELECT Indus, PERCENT_RANK() over(Order by Indus) p FROM boston)
    UNION ALL
    SELECT DISTINCT "30" as PERC_VAL, first_value(INDUS) OVER (Order BY CASE WHEN p<=0.3 THEN p END DESC) x1
    FROM(SELECT Indus, PERCENT_RANK() over(Order by Indus) p FROM boston)
    UNION ALL
    SELECT DISTINCT "40" as PERC_VAL, first_value(INDUS) OVER (Order BY CASE WHEN p<=0.4 THEN p END DESC) x1
    FROM(SELECT Indus, PERCENT_RANK() over(Order by Indus) p FROM boston)
    UNION ALL
    SELECT DISTINCT "50" as PERC_VAL, first_value(INDUS) OVER (Order BY CASE WHEN p<=0.5 THEN p END DESC) x1
    FROM(SELECT Indus, PERCENT_RANK() over(Order by Indus) p FROM boston)
    UNION ALL
    SELECT DISTINCT "60" as PERC_VAL, first_value(INDUS) OVER (Order BY CASE WHEN p<=0.6 THEN p END DESC) x1
    FROM(SELECT Indus, PERCENT_RANK() over(Order by Indus) p FROM boston)
    UNION ALL
    SELECT DISTINCT "70" as PERC_VAL, first_value(INDUS) OVER (Order BY CASE WHEN p<=0.7 THEN p END DESC) x1
    FROM(SELECT Indus, PERCENT_RANK() over(Order by Indus) p FROM boston)
    UNION ALL
    SELECT DISTINCT "80" as PERC_VAL, first_value(INDUS) OVER (Order BY CASE WHEN p<=0.8 THEN p END DESC) x1
    FROM(SELECT Indus, PERCENT_RANK() over(Order by Indus) p FROM boston)
    UNION ALL
    SELECT DISTINCT "90" as PERC_VAL, first_value(INDUS) OVER (Order BY CASE WHEN p<=0.9 THEN p END DESC) x1
    FROM(SELECT Indus, PERCENT_RANK() over(Order by Indus) p FROM boston)
    UNION ALL
    SELECT DISTINCT "100" as PERC_VAL, first_value(INDUS) OVER (Order BY CASE WHEN p<=1 THEN p END DESC) x1
    FROM(SELECT Indus, PERCENT_RANK() over(Order by Indus) p FROM boston)) ind_per
    ON nox_PER.PERC_VAL=ind_per.PERC_VAL

    """,
    con,
)

,PERC_VAL,NOX_PERC,INDUS_PERC
0,10,0.426,2.89
1,20,0.442,4.39
2,30,0.472,5.96
3,40,0.507,7.38
4,50,0.538,9.69
5,60,0.575,12.83
6,70,0.605,18.10
7,80,0.668,18.10
8,90,0.713,19.58
9,100,0.871,27.74


Показатели коррелируют между собой, коэффициент коррекляции близок к 1, прямая зависимость. Логически рост концентрации оксидов азота объясняется ростом промышленной застройки.